<a href="https://colab.research.google.com/github/Zack1243/Aniyomi-Downloader-through-links-file/blob/main/Aniyomi_local_anime_downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Libraries

In [37]:
import requests
import json
from google.colab import files
import os
from PIL import Image
import re
import subprocess
import sys
!python3 -m pip install -U --pre "yt-dlp[default]"
import yt_dlp
import zipfile

## Filter json file

In [38]:
def filter_details(file):

  # Get correct format options
  with open(file, 'r') as f:
    data = json.load(f)
    transformed_data = {
        "title": data.get("title", ""),
        "author": data.get("uploader", ""),
        "artist": data.get("uploader", ""),
        "description": data.get("description", ""),
        "genre": data.get("tags", []),
        "status": data.get("status", "0")
    }

    url_pattern = r'https?://\S+|www\.\S+'

    # Replace URLs with an empty string
    transformed_data["description"] = re.sub(url_pattern, '', transformed_data["description"])

    # Rewrite .json files
    with open(file, 'w') as f:
      json.dump(transformed_data, f)

## Extract url title and make directory for it

In [39]:
def downloader(url):
    try:
        # Get the video title
        result = subprocess.run([
            'yt-dlp',
            '--skip-download',
            '--get-title',
            url
        ], capture_output=True, text=True, check=True)

        # Extract the title from the result
        title = result.stdout.strip()

        # Make path to new directory including video's title
        path = os.path.join(os.getcwd(), title)

        # Check if path exists, else make one
        if not os.path.exists(path):
            os.makedirs(path)

        # Make a list for downloading the video
        subprocess.run([
            'yt-dlp',
            '--convert-thumbnails', 'jpg',
            '--progress',
            '--write-thumbnail',
            '--embed-thumbnail',
            '--add-metadata',
            '--write-info-json',
            '--remux-video', 'mkv',
            '-f', 'bestvideo[height<=1080]+bestaudio/best[height<=1080]/best',
            '-o', f'{path}/%(title)s.%(ext)s',
            url
        ], check=True)


        # Rename cover image to "cover.jpg"
        if os.path.exists(f'{path}/{title}.jpg'):
          os.rename(f'{path}/{title}.jpg', f'{path}/cover.jpg')
        else:
          print("No cover image found.")

        # Change info.json file to be "details.json"
        if os.path.exists(f'{path}/{title}.info.json'):
          os.rename(f'{path}/{title}.info.json', f'{path}/details.json')

          # Rewrite details.json to aniyomi style guidelines
          filter_details(f'{path}/details.json')
        else:
          print("No info file found.")

    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

## Example of aniyomi style .json file


```
{
	"title": "Example Title",
	"author": "Example Author",
	"artist": "Example Artist",
	"description": "Example Description",
	"genre": ["genre 1", "genre 2", "etc"],
	"status": "0",
	"_status values": ["0 = Unknown", "1 = Ongoing", "2 = Completed", "3 = Licensed", "4 = Publishing finished", "5 = Cancelled", "6 = On hiatus"]
}
```


## Zip and download files

In [40]:
def zip_and_download_subdirectories():
    # Get the current working directory
    current_directory = os.getcwd()

    # Get a list of all subdirectories
    subdirectories = [d for d in os.listdir(current_directory) if os.path.isdir(os.path.join(current_directory, d))]

    # Create a zip file
    with zipfile.ZipFile('anizip.zip', 'w') as zip_file:
        # Iterate through each subdirectory
        for subdirectory in subdirectories:
            # Get a list of all files in the subdirectory
            files_in_subdir = [os.path.join(subdirectory, f) for f in os.listdir(os.path.join(current_directory, subdirectory))]

            # Iterate through each file in the subdirectory
            for file_path in files_in_subdir:
                # Add the file to the zip file
                zip_file.write(os.path.join(current_directory, file_path), arcname=file_path)

    # Download the zip file
    files.download('anizip.zip')


## Get urls

In [43]:
if 'anizip' not in os.getcwd():
  try:
    os.mkdir(os.path.join(os.getcwd(), 'anizip'))
    os.chdir(os.path.join(os.getcwd(), 'anizip'))
  except:
    os.chdir(os.path.join(os.getcwd(), 'anizip'))

# Get urls from user
urls = input("Enter urls separated by space or enter: ")
urls = urls.split()

# Download each url
for url in urls:
  downloader(url)

# Zip files and download
zip_and_download_subdirectories()

Enter urls separated by space or enter: https://www.youtube.com/watch?v=nM06J9XwY8Y&list=RDMM&start_radio=1&rv=Dh262qUpcEY


KeyboardInterrupt: 

In [42]:
#TODO:
  # Check if correct resolution quality is being downloaded (<= 1080p)
  # Check if different sites meet the formatting guidelines
  # Check if aniyomi will accept these options